# PyCon 2017 ETL Workshop

## Table of Contents

* Test Optimizations
* References

In [11]:
import os
import subprocess
import shutil
import sys

ROOT_FOLDER = os.getcwd()
sys.path.append(os.path.join(ROOT_FOLDER, 'src'))

### Test Optimizations

Development turnaround can be shortened by optimizing repeated regression suite runs. In particular, You can avoid recreating a 'clean database' with every individual test case.

For most Always run all your SQLAlchemy tests inside a transaction:
* Provide a session fixture (SQLAlchemy `Session` instance) which is always rolled back.
* Using `.flush()` rather than `.commit()` to "persist" write operations.

This speeds up individual tests.  However, using `.flush()` should not be employed if your test actually needs to verify transaction rollback behavior.

One of the slowest steps is the test database setup.  By introducing a `--keepdb` option to our `pytest` suite, we can force a teamplate test database to be reused by:
* Setting the `base_dir` kwarg to a fixed path in the `testing.postgresql.Postgresql()` constructor.  
* Preventing re-initialization of the Postgres extensions, schemas and tables if the test database did not previously exists

This greatly reduces fixture setup time and allows us to immediately start running tests.  However, it is the responsibility of the developer to remove the test database if the model schema changes or a test accidentally persists a change to the database.

In [19]:
# no optimizations
%timeit -n1 -r3 subprocess.call('pytest src/', shell=True)

1 loop, best of 3: 2.73 s per loop


In [21]:
# with optimizations
import os
from src.tests.constants import KEEPDB_PATH

# ensure the first run creates the retained folder
testdb_path = os.path.join(ROOT_FOLDER, KEEPDB_PATH)
if os.path.exists(testdb_path):
    shutil.rmtree(testdb_path)

%timeit -n1 -r3 subprocess.call('pytest --keepdb src/', shell=True)

1 loop, best of 3: 895 ms per loop


## References

* [pytest](https://docs.pytest.org/en/latest/contents.html)
* [testing.postgres](https://github.com/tk0miya/testing.postgresql)
* [Jupyter Notebook](http://jupyter.org/)
    * [Cell Magics](https://ipython.org/ipython-doc/3/interactive/magics.html#cell-magics)